##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# チェックポイント保存の移行

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/checkpoint_saver">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/checkpoint_saver.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>


継続的に「最適な」モデルまたはモデルの重み/パラメータを保存することにより、トレーニングの進行状況を追跡したり、保存されたさまざまな状態から保存されたモデルを読み込んだりできます。

TensorFlow 1 で `tf.estimator.Estimator` API を使用してトレーニング/検証中にチェックポイントの保存を構成するには、`tf.estimator.RunConfig` でスケジュールを指定するか、`tf.estimator.CheckpointSaverHook` を使用します。このガイドでは、このワークフローから TensorFlow 2 Keras API に移行する方法を実演します。

TensorFlow 2 では、さまざまな方法で `tf.keras.callbacks.ModelCheckpoint` を構成できます。

- `save_best_only=True` パラメータを使用して監視された指標に従って、「最適な」バージョンを保存します。ここで、`monitor` は、たとえば `'loss'` 、`'val_loss'`、`'accuracy'`、または 'val_accuracy'` などになります。
- `save_freq` 引数を使用して特定の頻度で継続的に保存します。
- `save_weights_only` を `True` に設定して、モデル全体ではなく重み/パラメータのみを保存します。

詳細については、`tf.keras.callbacks.ModelCheckpoint` API ドキュメントと、[モデルの保存と読み込み](../../tutorials/keras/save_and_load.ipynb)チュートリアルの*トレーニング中にチェックポイントを保存する*セクションを参照してください。チェックポイント形式の詳細については、[Keras モデルの保存と読み込み](https://www.tensorflow.org/guide/keras/save_and_serialize)ガイドの *TF チェックポイント形式*セクションを参照してください。さらに、フォールトトレランスを追加するには、`tf.keras.callbacks.BackupAndRestore` または `tf.train.Checkpoint` を手動チェックポイントに使用できます。詳細は、[フォールトトレランス移行ガイド](fault_tolerance.ipynb)を参照してください。

Keras [コールバック](https://www.tensorflow.org/guide/keras/custom_callback)は、組み込みの Keras `Model.fit`/`Model.evaluate`/`Model.predict` API のトレーニング/評価/予測時のさまざまな時点で呼び出されるオブジェクトです。詳細は、ガイドの最後の *次のステップ*セクションを参照してください。

## セットアップ

まず、インポートし、デモ用の単純なデータセットから始めます。

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: tf.estimator API でチェックポイントを保存する

次の TensorFlow 1 の例は、`tf.estimator.RunConfig` を構成し、`tf.estimator.Estimator` API を使用してトレーニング/評価中にすべてのステップでチェックポイントを保存する方法を示しています。

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%ls {classifier.model_dir}

## TensorFlow 2: Model.fit の Keras コールバックでチェックポイントを保存する

TensorFlow 2 では、組み込みの Keras `Model.fit`（または `Model.evaluate`）をトレーニング/評価に使用する場合、`tf.keras. callbacks.ModelCheckpoint` を作成し、それを `Model.fit`（または `Model.evaluate`）の `callbacks` パラメータに渡します。（詳細については、API ドキュメントと、[組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)ガイドの*コールバックの使用*セクションを参照してください。）

以下の例では、`tf.keras.callbacks.ModelCheckpoint` コールバックを使用して、チェックポイントを一時ディレクトリに保存します。

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=log_dir)

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[model_checkpoint_callback])

In [ ]:
%ls {model_checkpoint_callback.filepath}

## 次のステップ

チェックポイントの詳細については、次を参照してください。

- API ドキュメント: `tf.keras.callbacks.ModelCheckpoint`
- チュートリアル: [モデルの保存と読み込み](../../tutorials/keras/save_and_load.ipynb)（*トレーニング中にチェックポイントを保存する*セクション）
- ガイド: [Keras モデルの保存と読み込み](https://www.tensorflow.org/guide/keras/save_and_serialize)（*TF チェックポイント形式* セクション）

コールバックの詳細については、次を参照してください。

- API ドキュメント: `tf.keras.callbacks.Callback`
- ガイド: [コールバックを記述する](https://www.tensorflow.org/guide/keras/guide/keras/custom_callback)
- ガイド: [組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)（*コールバックの使用*セクション）

次の移行関連のリソースも参照してください。

- [フォールトトレランス移行ガイド](fault_tolerance.ipynb): `Model.fit` 用の `tf.keras.callbacks.BackupAndRestore`、または、カスタムトレーニングループ用の `tf.train.Checkpoint` および `tf.train.CheckpointManager` API
- [早期停止移行ガイド](early_stopping.ipynb): `tf.keras.callbacks.EarlyStopping` は組み込みの早期停止コールバックです
- [TensorBoard 移行ガイド](tensorboard.ipynb): TensorBoard により、指標の追跡と表示が可能になります
- [LoggingTensorHook と StopAtStepHook から Keras コールバックへの移行ガイド](logging_stop_hook.ipynb)
- [SessionRunHook から Keras コールバックへのガイド](sessionrunhook_callback.ipynb)